In [13]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import gradient_descent_v2
import random



In [14]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('D:/Chatbot/Simple-Python-Chatbot-master/intents.json', encoding='utf8' ).read()
intents = json.loads(data_file)

for intent in intents['intents']:
    #print(intent['patterns'])
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        # print(pattern)
        # print(w)        
        words.extend(w)
        # print(words)
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

print(len(words))

words = sorted(list(set(words)))

print (len(words), "unique lemmatized words", words)
classes = sorted(list(set(classes)))

# print (len(documents), "documents",documents)

print (len(classes), "classes", classes)

# print (len(words), "unique lemmatized words", words)


# pickle.dump(words,open('D:/Chatbot/Simple-Python-Chatbot-master/words.pkl', 'wb'))
# pickle.dump(classes,open('D:/Chatbot/Simple-Python-Chatbot-master/classes.pkl', 'wb'))



313
100 unique lemmatized words ["'s", ',', '.', ';', 'alo', 'awesome', 'bao', 'biệt', 'bye', 'bênh', 'bạn', 'bệnh', 'bị', 'bựa', 'ca', 'chatting', 'cho', 'chào', 'chết', 'chữa', 'có', 'công', 'cảm', 'cộng', 'của', 'cứu', 'day', 'dịch', 'dụng', 'em', 'for', 'giúp', 'good', 'goodbye', 'gì', 'hello', 'helpful', 'helping', 'hey', 'hola', 'hình', 'hôm', 'hợp', 'không', 'khỏe', 'khỏi', 'later', 'là', 'làm', 'lây', 'me', 'mất', 'mấy', 'mệt', 'mỏi', 'mục', 'nam', 'nay', 'next', 'ngày', 'người', 'nhiêu', 'nhiễm', 'nhọc', 'ni', 'nice', 'nào', 'ok', 'ra', 'rồi', 'see', 'số', 'thank', 'thanks', 'that', 'thấy', 'thế', 'thể', 'till', 'time', 'to', 'trường', 'trị', 'tình', 'tôi', 'tạm', 'tổng', 'việt', 'vào', 'xin', 'you', 'đi', 'đã', 'đích', 'được', 'đỡ', 'ơi', 'ơn', 'ạ', 'ở']
8 classes ['goodbye', 'greeting', 'noanswer', 'options', 'thanks', 'tired', 'today', 'totalvn']


In [15]:
# initializing training data
training = []
output_empty = [0] * len(classes)
#print(documents)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    #print(pattern_words)
    #lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        #print(w)
        bag.append(1) if w in pattern_words else bag.append(0)
    #print(bag)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    #print(training)

#shuffle our features and turn into np.array
random.shuffle(training)
print(type(training))
training = np.array(training)
# # create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])

train_y = list(training[:,1])
print(np.shape(train_x))
print("Training data created")

<class 'list'>
(77, 100)
Training data created


c:\Users\Admin\anaconda3\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [26]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = gradient_descent_v2.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
#model.save('D:/Chatbot/Simple-Python-Chatbot-master/chatbot_model100.h5', hist)

print("model created")

c:\Users\Admin\anaconda3\envs\Tensorflow\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:358: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               12928     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 520       
Total params: 21,704
Trainable params: 21,704
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
16/16 [==============================] - 2s 2ms/step - loss: 2.1130 - accuracy: 0.0979 
Epoch 2/200
16/1

In [27]:
y_pred = model.predict([train_x])